In [3]:
#!pip install pyodbc

                                              0.0/69.7 kB ? eta -:--:--
     -----                                    10.2/69.7 kB ? eta -:--:--
     -----                                    10.2/69.7 kB ? eta -:--:--
     -------------------------------------- 69.7/69.7 kB 475.4 kB/s eta 0:00:00


In [1]:
# Import packages
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.0f' % x)

import pyodbc
import numpy as np
import warnings
import pickle
import re

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
warnings.filterwarnings("ignore")

# Query SQL
query = """
Select distinct ci.NoKK , a.SK_Created_Date, de.NIK, ci.CustomerID, ci.FullName As [customer_name]
from DWBIBFI2_DWH.dbo.fact_udp_sales_detail a (nolock)
Left Join Risk.dbo.Customer_Information ci (nolock) On a.ID_Application = ci.applicationid
Left Join DWBIBFI2_DWH.dbo.Fact_contract fc (nolock) On a.ID_Application = fc.id_application
Left Join DWBIBFI2_DWH.dbo.Dim_Employee de (nolock) On fc.SK_Marketing_3 = de.SK_Employee
Where a.SK_SOA = 17 And a.SK_Customer_Status in (2, 3) And a.SK_Product in (4, 6) And a.SK_Activity = 12
"""

In [2]:
query = """
With
customer_relative as (
    Select 
        a.SK_Product, 
        ci.NoKK, 
        fc.AgreementNo, 
        fc.SK_GoLive_Date,
        dcs.Customer_Status,
        Row_Number() Over(Partition By a.SK_Product, ci.NoKK, fc.AgreementNo, dcs.Customer_Status 
        Order By fc.SK_GoLive_Date) as seq
    From
        DWBIBFI2_DWH.dbo.fact_udp_sales_detail a (nolock)
    Left Join
        DWBIBFI2_DWH.dbo.Fact_contract fc (nolock) On a.ID_Application = fc.id_application
    Left Join
        Risk.dbo.Customer_Information ci (nolock) On a.ID_Application = ci.applicationid
    Left Join
        DWBIBFI2_DWH.dbo.Dim_Customer_Status dcs (nolock) On a.SK_Customer_Status = dcs.SK_Customer_Status 
    Where
        a.SK_SOA = 17
    And
        a.SK_Customer_Status in (2, 3)
    And
        a.SK_Product in (4, 6)
    And
        fc.SK_GoLive_Date is not null
    And
        ci.NoKK is not null
)
Select *
From customer_relative 
Where seq = 1;
"""

In [3]:
# set up connection
DRIVER = ""
driver_names = [x for x in pyodbc.drivers() if x.endswith('SQL Server')]
if driver_names:
    DRIVER = driver_names[0]
SERVER = "172.16.101.164"
DATABASE = "DWBIBFI2_DWH"
USER = "Data_Inovasion"
PASSWORD = "Inovasion999"

if DRIVER:
    conn_string = "Driver="+DRIVER+";Server="+SERVER+";Database="+DATABASE+";UID="+USER+";PWD="+PASSWORD+";Trusted connection=YES;"
    conn = pyodbc.connect(conn_string)
    print('Suitable driver found. Connected')
else:
    print('(No suitable driver found. Cannot connect.)')

Suitable driver found. Connected


In [4]:
nokk = pd.read_sql(query, conn)

In [7]:
nokk.to_csv('data_baru.csv', index=False)

In [8]:
del nokk

In [25]:
nokk = pd.read_csv('data_baru.csv')
nokk['is_null'] = nokk.NoKK.isnull()
nokk['NoKK']=nokk['NoKK'].astype(str)
nokk['NoKK']= nokk['NoKK'].str.replace('+','')
nokk['NoKK']= nokk['NoKK'].str.replace('.','')
nokk['NoKK']= nokk['NoKK'].str.replace('`','')
nokk['NoKK']= nokk['NoKK'].str.replace('-','')
nokk['NoKK']= nokk['NoKK'].str.replace('/','')
nokk['NoKK']= nokk['NoKK'].str.replace(' ','')
nokk.head()

,SK_Product,NoKK,AgreementNo,SK_GoLive_Date,Customer_Status,seq,is_null
0,4,1271150908060002,4011903769,20190809,New,1,False
1,4,1271150908100001,4011601058,20160324,New,1,False
2,4,1271151012070002,4011905140,20191021,New,1,False
3,4,1271151110050002,4012201161,20220209,New,1,False
4,4,1271151710060003,4011700079,20170112,New,1,False


In [26]:
r = re.compile(r'^\W+$') 
nokk['special_char_only'] = nokk.NoKK.apply(lambda x: bool(r.match(x)))

w = re.compile(r'^ +| +$')
nokk['have_leading_white_space'] = nokk.NoKK.apply(lambda x: bool(w.match(x)))
nokk['have_trailing_white_space'] = nokk.NoKK.apply(lambda x: bool(x.endswith(' ')))
nokk['have_trailing_special_char'] = nokk.NoKK.apply(lambda x: bool(x.endswith('`')) or bool(x.startswith('+')) or bool(x.startswith('.')))

nokk['is_numeric'] = nokk.NoKK.str.isnumeric()

nokk['len'] = nokk['NoKK'].apply(len)
nokk.head()

,SK_Product,NoKK,AgreementNo,SK_GoLive_Date,Customer_Status,seq,is_null,special_char_only,have_leading_white_space,have_trailing_white_space,have_trailing_special_char,is_numeric,len
0,4,1271150908060002,4011903769,20190809,New,1,False,False,False,False,False,True,16
1,4,1271150908100001,4011601058,20160324,New,1,False,False,False,False,False,True,16
2,4,1271151012070002,4011905140,20191021,New,1,False,False,False,False,False,True,16
3,4,1271151110050002,4012201161,20220209,New,1,False,False,False,False,False,True,16
4,4,1271151710060003,4011700079,20170112,New,1,False,False,False,False,False,True,16


In [27]:
nokk.loc[(nokk['is_numeric'] == True) & (nokk['len'] == 16), 'remarks'] = 'Valid' 
nokk.loc[(nokk['is_numeric'] == True) & (nokk['len'] == 17), 'remarks'] = 'Valid' 
nokk.loc[(nokk['is_numeric'] == True) & (nokk['len'] > 17), 'remarks'] = 'Not Valid : Digit > 17' 
nokk.loc[(nokk['is_numeric'] == True) & (nokk['len'] < 16), 'remarks'] = 'Not Valid : Digit < 16'  
nokk.loc[nokk["is_null"] == True, 'remarks'] = 'Not Valid : Null / Blank'
nokk.loc[nokk["special_char_only"] == True, 'remarks'] = 'Not Valid : Special character'
nokk.loc[(nokk['have_leading_white_space'] == True) | (nokk['have_trailing_white_space'] == True), 'remarks'] = 'Not Valid : Have_white_space' 
nokk.loc[(nokk['is_numeric'] == False) & (nokk['is_null'] == False) & (nokk['special_char_only'] == False)
           & (nokk['have_leading_white_space'] == False) & (nokk['have_trailing_white_space'] == False), 'remarks'] = 'Not Valid : Alpha numeric'  

nokk_clean = nokk[['NoKK' ,"remarks"]]
nokk_clean['NoKK_clean'] = nokk_clean['NoKK'].apply(lambda x: x.strip())
nokk_clean.head()

,NoKK,remarks,NoKK_clean
0,1271150908060002,Valid,1271150908060002
1,1271150908100001,Valid,1271150908100001
2,1271151012070002,Valid,1271151012070002
3,1271151110050002,Valid,1271151110050002
4,1271151710060003,Valid,1271151710060003


In [28]:
nokk.loc[nokk['remarks'] == 'Not Valid : Alpha numeric']

,SK_Product,NoKK,AgreementNo,SK_GoLive_Date,Customer_Status,seq,is_null,special_char_only,have_leading_white_space,have_trailing_white_space,have_trailing_special_char,is_numeric,len,remarks
93,4,1271KW270920170007,4011905901,20191126,New,1,False,False,False,False,False,False,18,Not Valid : Alpha numeric
94,4,1271KW270920170007,4011905902,20191126,New,1,False,False,False,False,False,False,18,Not Valid : Alpha numeric
106,4,,4122001126,20200312,New,1,False,False,False,False,False,False,0,Not Valid : Alpha numeric
107,4,,4322100394,20210312,New,1,False,False,False,False,False,False,0,Not Valid : Alpha numeric
108,4,,4811907236,20191231,New,1,False,False,False,False,False,False,0,Not Valid : Alpha numeric
109,4,,5271600435,20160608,New,1,False,False,False,False,False,False,0,Not Valid : Alpha numeric
110,4,,5321700058,20170310,New,1,False,False,False,False,False,False,0,Not Valid : Alpha numeric
111,4,,5321700340,20171117,New,1,False,False,False,False,False,False,0,Not Valid : Alpha numeric
112,4,,5381801215,20180831,New,1,False,False,False,False,False,False,0,Not Valid : Alpha numeric
113,4,,5701700553,20170719,New,1,False,False,False,False,False,False,0,Not Valid : Alpha numeric


In [29]:
nokk.loc[nokk['have_trailing_special_char'] == True]

,SK_Product,NoKK,AgreementNo,SK_GoLive_Date,Customer_Status,seq,is_null,special_char_only,have_leading_white_space,have_trailing_white_space,have_trailing_special_char,is_numeric,len,remarks


In [30]:
result = (nokk.groupby(['remarks']).size()
   .sort_values(ascending=False) 
   .reset_index(name='count') )
   #.drop_duplicates(subset='len'))
print(result)

                     remarks   count
0                      Valid  144895
1     Not Valid : Digit < 16    2930
2  Not Valid : Alpha numeric    1061
3     Not Valid : Digit > 17     161


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   col1    5 non-null      int64 
 1   col2    5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes
